# https://www.kaggle.com/datasets/mdabdullahalmashud/medical-audio-50

In [1]:
# prompt: import mp3 folder

import os
import librosa
import numpy as np

def import_mp3_folder(folder_path):
  """
  Imports all MP3 files from a specified folder and returns a dictionary
  containing the file name and the audio data.

  Args:
      folder_path: The path to the folder containing the MP3 files.

  Returns:
      A dictionary where keys are file names and values are audio data
      loaded using librosa.
  """

  audio_data = {}
  for filename in os.listdir(folder_path):
    if filename.endswith(".mp3"):
      file_path = os.path.join(folder_path, filename)
      try:
        audio, sr = librosa.load(file_path)
        audio_data[filename] = audio
      except Exception as e:
        print(f"Error loading {filename}: {e}")

  return audio_data

folder_path = 'mp3'
audio_dict = import_mp3_folder(folder_path)

<ipython-input-1-16af269a2c03>:25: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load(file_path)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


In [2]:
!pip install pydub

In [3]:
from pydub import AudioSegment
import os

def convert_mp3_to_wav(mp3_folder, wav_folder):
    """Convert all mp3 files in a folder to wav format."""
    if not os.path.exists(wav_folder):
        os.makedirs(wav_folder)

    for filename in os.listdir(mp3_folder):
        if filename.endswith('.mp3'):
            mp3_path = os.path.join(mp3_folder, filename)
            wav_path = os.path.join(wav_folder, filename.replace('.mp3', '.wav'))

            # Load and convert the mp3 file to wav
            audio = AudioSegment.from_mp3(mp3_path)
            audio.export(wav_path, format="wav")

# Example usage
mp3_folder = 'mp3'
wav_folder = 'wav'
convert_mp3_to_wav(mp3_folder, wav_folder)


In [4]:
!pip install openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 18.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.8 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803321 sha256=d54cc7d44b9dcca705ae91515753be92a0f4da20f9f97b618c88c200af6b5668
  Stored in directory: /root/.cache/pip/wheels/dd/4a/1f/d1c4bf3b9133c8168fe617ed979cab7b14fe381d059ffb9d83
Successfully built openai-whisper


In [5]:
import whisper
import os

def transcribe_audio_files(wav_folder, transcription_folder):
    """Transcribe WAV files in a folder using Whisper model."""
    if not os.path.exists(transcription_folder):
        os.makedirs(transcription_folder)

    # Load Whisper model
    model = whisper.load_model("base")

    for filename in os.listdir(wav_folder):
        if filename.endswith('.wav'):
            wav_path = os.path.join(wav_folder, filename)
            result = model.transcribe(wav_path)

            # Save the transcription as a text file
            transcription_file = os.path.join(transcription_folder, filename.replace('.wav', '.txt'))
            with open(transcription_file, 'w') as f:
                f.write(result['text'])

# Example usage
transcription_folder = 'transcriptions'
transcribe_audio_files(wav_folder, transcription_folder)

100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 134MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

In [6]:
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 28.2 MB/s eta 0:00:00


In [20]:
import jiwer
import os
import time  # For calculating RTF
from jiwer import measures  # Import jiwer.measures for computing measures


def calculate_kpis(transcription_folder, reference_transcripts):
    """Calculate WER, CER, RTF, accuracy, precision, recall, f1 score, deletion rate, insertion rate."""

    kpi_results = {
        "wer": [],
        "cer": [],
        "rtf": [],
        "accuracy": [],
        "precision": [],
        "recall": [],
        "f1_score": [],
        "deletion_rate": [],
        "insertion_rate": []
    }

    # Iterate over files in the transcription folder (predicted transcripts)
    for filename in os.listdir(transcription_folder):
        if filename.endswith('.txt'):
            transcription_file = os.path.join(transcription_folder, filename)

            # Read the predicted transcript from the folder
            with open(transcription_file, 'r', encoding='utf-8') as f:
                predicted_transcript = f.read()

            # Get the corresponding reference transcript from the variable
            file_key = filename.replace('.txt', '')  # Assuming the reference transcript key matches the file name
            if file_key in reference_transcripts:
                reference = reference_transcripts[file_key]
            else:
                print(f"Reference transcript for {file_key} not found.")
                continue

            # Calculate WER, CER, precision, recall, and F1 using jiwer with transformations
            start_time = time.time()  # Start timer for RTF

            wer = jiwer.wer(reference, predicted_transcript)
            cer = jiwer.cer(reference, predicted_transcript)

            # End timer for RTF and calculate elapsed time
            elapsed_time = time.time() - start_time

            # Check if predicted_transcript is empty before calculating RTF
            if len(predicted_transcript) > 0:
                rtf = elapsed_time / (len(predicted_transcript) / 100)  # Assuming 100 characters per second
            else:
                rtf = 0  # or another appropriate value, like float('inf')

            # Calculate measures using jiwer.measures.compute_measures
            measures_result = measures.compute_measures(reference, predicted_transcript)

            # Store the results
            kpi_results["wer"].append(wer)
            kpi_results["cer"].append(cer)
            kpi_results["rtf"].append(rtf)
            kpi_results["accuracy"].append(1 - wer)  # 1 - Word Error Rate (WER)
            kpi_results["precision"].append(measures_result['wip'])  # Word Information Preserved
            kpi_results["recall"].append(measures_result['wil'])  # Word Information Lost
            kpi_results["f1_score"].append(measures_result['mer'])  # Match Error Rate
            kpi_results["deletion_rate"].append(measures_result['deletions'] / (measures_result['substitutions'] + 1e-10))
            kpi_results["insertion_rate"].append(measures_result['insertions'] / (measures_result['substitutions'] + 1e-10))

    # Aggregate KPIs (calculate average) - Add check for empty lists
    avg_kpis = {}
    for k, v in kpi_results.items():
        if v:  # Check if the list is not empty
            avg_kpis[k] = sum(v) / len(v)
        else:
            avg_kpis[k] = 0  # Or another default value like float('nan')

    return avg_kpis

# Example reference transcripts (stored in a dictionary variable)
reference_transcripts = {
    'T__LifeSpring_1': 'Қaatashime galo international mainstream hygiene de mainstream hygiene bol tashu lam reki bhochi 8 intashu le kubi gurutta purnu amade deshi jyodyo oha bejini shi te re aalachuna hoa na kintu masi ke shomuay pori chanata maintain kora masi ke shushas te jebapatta 8 intashu le kubi gurutta purnu ʻum dhiyajet tein shale mainstream hygiene de muil thein te itchidoeta jay make mainstream and normal fact of life by 2030 8 intashu le kubi ujta purnu ʻum dhiyajet tein shale māsik jyegte nāri riktu tun to shaba bhi khatu na e i i i i shtak shaba mute jukeli dāve ʻasule itke kyanate bhi gurutta dāgut se itke etta shādharun khatu na hinsi be manush keksab karatunu e i pitsunā shu te unik gurukaruna tse amade deshi be bhi nnu elakai nāri dhe kamra dekhi je shaba itua shule shaba nbabe privilits known shaba i shaba n shu ju shubhi dha pannab hādhar shādharun khali amade deshi je jyegte nāri dhe paird bhebhar kutta bātschen afford kutta bātschen tarfamli tse teke pozitivli shaba bhi dha ho tse jinistā kinda oning jyagrātshe vind shādharun anjule amade deshi je nāriya kindu maasik ki e bhebapat teneye kubakta shādharun tun nārn e vung tāne bhebapat teke mone kauranjie amade bhebapat tene kauranjie kauratha bhaul tepar buna at bhi jyegte kubkhara paktat tain e i shumaiteni alaqona kunu bhabi kaura ujitna e dhe jyegte dhe khadar je maasikahatta paird lag bheb poli bhebharia hādda baba upadjun kama aptake bholte bātschen na tini affustivod kutchen ibu ar aftur tse jinistā tini kubshaba bhi bhabi prokash kurtta bārana ibu e dhe jyegte hodse dhe khadar tse jitāra affustivod kaur kapur bhabharkutchen tula bhabharkutchen ibu shabchidu kho janojie kapur kunu dhe khadar jie bhar kaurin killed가락 trandehto sade turn Technologies pand bush tested jyegte baatzha cabar par clouds Warja maasikhi sube bliding tuther i 52finnidaf 53finnidafeliolution bachelor effective studies Massachusetts ھا پشو مراشنگیدی ایک تشنیش چالکوری دیکھون جی جو دی امرہ پوری چھانوں بھابیں ماشی کے شومتہ پانھ نہیں کری امر جیتی تنی شوچی تو نہ ہوئی ابیپر جیتے میں شاہدہ رون بھابیں نانی ایک تنی میی جو دیتر مارکی شوچھ پہے بلتنا پرے شیجو دی پیڑی جونوں تر بابا کے بلتنا پرے جی پہ با آفیسٹی کاشش ماما جونے تو پیڑی نیشو تا لیکن دو تارجونوں ایماشی کے شومتہ کو کاشتو کر ہوئی دارا ایبو امرہ ایت او شونی جو کنہ امرہ پیشندے کے بولی این گاٹیانہ بولن جی امرہ مامی تو کافر بابا کریں شامل کنو شامو شامی او بابا کر لیکن شامو شامی اشونی دیکھون شامو شاہدی تو امرہ نوتنوتنی جانتے پریبا شکتے پری ایبو ان دیکھا جی ایزی ایماشی کے شومائی پوری چھانوں تھا اٹتد مینسٹرال حاجین شامو شاہدی تو ایسے ایسیونہ امی شاموستو عجارت کی شوری جارا نوتن ماشیجا دیشورو ہوتشے بابا مہیشب اپنا دیجا مان دائکتو دیکھا ماشی کے بیپر گولے بوچی دا ایک بھابی ایماشیجتے جی کب شادہا نکتا گھتونہ ایتج پر اتکتی ناری جبونے ایتی نیدیشتو شامویت ہوای ماشیجا مدر شاریلی بھا بیک جون ناری تار کنو اوشوی تا تکینا دیکھا ماشیج اونیمی دکھولے اتھو ماشیگ نہ ہو لے اتھو اونےگ بیشیورے کنطام نچینتی تو ہو جی تر اٹھو کی ایت شوریر امین اکت جینی امین اکت گھتونہ جیت ماشیماشا ہون دیکھ جانان دائی جہا اما شوریشب سستمگلو شرنان مدتو چاہلچے اتبعی جینیشتے کب شاب بابی اگھا مہنو اکن دا کن اشوری جو گے دا پیپر اما ای بولشی نا ایتنی اپنے کلکولی اعلوچونہ کلون ایتا که ایمپورٹن ایک تا انوچونہ توپیک سوک کین تو کچھ شاہدھا اون بھابے ایک جون ناری تر ماشیک اشومت تار ایجیت ٹائمتا اییت ٹائمتا کیشی جانانکت شاہدھا اون اشومت ایشبی نیتا پری شیجنو تر فیم لیکچھے سپوچ چیتا پری شیجنو تر با با اطبع حزبین اطبع بہایتا که بولت پر جامر پید لگ بئینتا ایوین امدی سکون کلکولیج کلوتے بھی نا جا گئی اماشیگی شو مائی پسانیشری پیدر جون جو دی کونسستم راکھا جائے ایجی اام را جتا اچھی جی اام راک کون کون بچاکا دی شیمیشن تتک امنا پیکٹنی تی پاچھی شیتا کین تو امدیشن اارو بھالو ایو پس پسا ایت گلت پر نو بیشا شیتا اوچھی ایوین امدیشن ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایو surgeries ,یوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایو une ایوی dieser ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوzilیوین ایوین ایوین ایوین ایوین ایوین ایوین مع ایوون ایو',
    'T__LifeSpring_2': 'Amra, aeportjunctu, onigrokomir o cd, theme, but content, bapari amra ishune chtainu, jhe mon jiba no de kreena pae, sapi chunapak mo nehae, aeko tamatai to klesha hojibbir ho te chana. Innes ni ye shara kondakha je je intrusive thought ho de tha ke, dar pre ho chhe je sexual type of content frequently matam mo te ashe. Ae golo bapari amra ki to kongbe si bhalui jani tainu. Ae kia mi, ae reig dharo ne o cd veritine ye kichu kata bol bu, shaita ho chhe je agressive variety of OCD. Ita ki jinish, dakin nami ho chhe amra to dharo napat chtainu je agressive variety. Ki ho ye kane je oi partikular patiente mo nehae tor teke, ekthana aggrishen kach ko re, ki rokom aggrishen. Ita kindu kubi asole distressing e bom disrap deep type. Ki ho ye kane. Ammi asole ae gokom patientke, onig bhi si manoshig habe jontrona pete de ki. Onig bhi kosh te tha ke, onig worried tha ke, onig angzras tha ke. Ki ho ye, dhaka jai ke, ce oi partikular patiente mo nekori ki, ce amar madhu ne, amar kachir manushamar puri barer konon log jon, amar konu friend, athaba ho chhe bilavit konu person ke, ammi khote kore feldepari. Onig bhi si rokomir haam full ae kichu ghoti hai feldepari, jar karneshie, dhur kata nae, putitu ho te pari, athaba ho chhe je she even marau jete pari. Ae jin bhoi da, je ita tar muthe kach kore ke na je she kindu, jani je ami chai le, kindu tar khote kore feldepari. Ae bhoi da, kindu, e bhoi da ita ar muthe kore kore kore kai, je she shara konu jani wori takin, barer ni jir upothe ke, sheta niyum chon hari feldepari, jikunu shama shhe, ae munekta arj fil kore abar bhoi opai, jar amar tanataru khote kore jete pari. Ae jee ae ta, lup, e lupir mo de, oi be chara ba be chari, ghor pa kete tha ke, e bhoi na ko rakumir tha tha ke, je supris kore kje no, banu jo kore kje no, aning bhi shye mental iford de, ae visha saikel ta, roti din, roti dinyotu tar muthe, tine bhi shilbhak shomuay ghotte tha ke, e bung da kha jai, je uiter fale, oi be chai, aning bhi shi wori takin, e bhoi anjas tha ke, je je agrisi variety of OCD, eta tine torneer hoe tha ke, tine torne bhoi de, tine torneer, tine hoe tha ke, bhi bhi no gavisyonay da kha jai, shite kiki je, harming aadars, eta kiro kum, e jai kto aage jeta bholam, shi chintakorikje, amar ghorre manush ke ami jai kuno shomuay myri fil te pari, kono chhuri dikle tar mone hoi ki, ja ami akhan haudibat, amar shamne manushta ke, stebh kore fil te pari, tar diya shlai dikle mone hoi ki, ja ami ghorre bhi tar agun lagi fil te pari, hae ba ami amar shamne manushta gae agun lagi di te pari, eidhaar onir bhoi ta tar muthe kach kurte tha ke, shi gintakasule kore fil le na, hae ba, kintu shi oibhoi ta tar muthe kach kurikje, ami amar kontri ni jero po kontrulta ke hari fil te pari, ebong ami ita chaili purte pari, ae je, thinking termode, etaar prosister mode shi gurthe tha ke, ditio je pattern da shita ho che, je, hari fik, ebong ho che, dinjara stebh of je, e mei jes gulho, at kalpona jitra gulho, tar chokhi shamne bha shte tha ke, abar tari, disease prosister karo ne, shi chok bontho kore priparasi ni, kaalpona kurthe tha ke, je, shi tar, kachil kono manushke, ba, tar puri barel logjon ke, ba, tar friend huk, je, huk karo, che kote kore fil te pari, ki, rokom shi kaalpona kurthe tha ke, ja, ami, ami, ami gammar kono friendesha de, ami, roth kraskurt shi, e shamne ami, oke, dhakkadi a, ami fil le dilam, roth er mode hai, ek tan gari aur hupudde, uthe chole ja, eidhanir, thot termode, ashtethe ke, thot ki, shi kaalpona kore, abar ki ashe, irkomu dhakkha ja, je, no tun ke, ma, huye, che, shi tar, bebi ke, chole fil te, beba, che, tar, bebi ke, ami, abe, che, pe, durbe, ja, na, bebi tar, ek tan, kankhi to, ba, aur pruttashi to, ba, aur aur akumir, konu, dur khatunak hurthe ja, abar karo, karo, khi, to, irkomu dhakkha ja, je, tar, ashust, to, ba, ba, ba, ba, ba, ek dung, bea, chanae, poria, se, mu, mu, shawbos, tha, ta, dhe, ke, jo, shi, mo, ne, kore, ki, jami, ba, list, cha, padhe, mere, fili, ma, e, rokom, thot, ne, o, kin, tu, ashe, ma, prochon, oye, be, charabab, be, chari, ashole, prochon, no, rokomir, mentally, distress, ni, ashe, jami, tu, itas, le, kurthe, cha, i, na, kin, tamar, be, dur, itas, chole, ashust, shi, tri, tu, je, petan, shi, tu, je, doing something, embarrassing, but illegal. shi, tu, ki, rokom, je, ashole, shi, ka, ki, opuman, kurthe, cha, tine, kin, tu, hoot, kore, kono rokomir, kono, prohokir, kasi, chara, shi, tar, sham, ne, raktum, manu, shi, a, jo, ta, i, shi, opuman, kore, fil, shi, na, to, opuman, shi, a, kurthe, to, bha, shai, shi, ga, legala, chat, ba, kore, di, jo, ma, emun, embarrassing, ki, chuk, kore, fili, j, ta, pore, koni, ta, lo, kachi, ma, ne, ho, chai, ta, amiku, bha, ember, s, fil, kurthe, bom, j, kye, shi, tar, ne, kono, kata, boli, fili, shi, da, kha, j, j, embarrassing, fil, kore, i, je, petan, gullu, amiku, boli, ha, j, aj, aj, aj, tine, gullu, eg, gullu, patient, kin, tu, emun, bha, na, j, tar, bha, v, v, j, tar, kono, violence, sir, kono, iti, hash, aj, aj, e, ja, ho, chat, kore, i, shul, ho, shi, adris, kurthe, pare, j, amar, e, je, thart, gullu, ash, che, eg, gullu, o, proje, ni, ho, j, chit, ho, o, he, tu, emun, amar, j, ho, te, pare, e, bom, ordnir, j, no, rish, ge, ho, te, pare, shi, eg, gullu, adris, te, ke, kurthe, pare, k, n, tu, shi, ait, a, te, ke, bhe, rote, chai, ho, k, n, tu, shi, ait, a, te, ke, bhe, rote, pare, na, on, he, bhe, rote, ge, ar, ch, fil, kore, i, tai, ho, te, ge, kore, j, dhar, on, adr, ar, e, jo, no, patient, kikore, j, bhe, bhe, nor, akomir, j, thart, gullu, ni, a, che, tar, e, thart, gullu, k, che, on, he, replace, korar, che, stakore, e, bom, shi, eg, la, ke, sap, pris, korar, che, stakore, abar ho, che, on, he, bhe, rote, pare, ho, te, pare, ho, te, na, e, bom, ordnir, j, ni, ne, j, ke, no, ash, li, bha, most commonly, dhaka, j, j, j, n, tika, j, influence, sheta, to, a, che, tar, pore, dhaka, j, oi, particula, patient, ter, j, di, friend, per, family, nir, kar, mo, dhe, j, di, o, c, di, tha, ke, tha, ke, topon, dhaka, j, che, ch, ta, kor, e, bhe, korar, e, kore, j, amar, j, bila, ve, tar, ho, bha, ma, family, m, ba, frender, j, o, c, di, ta, a, che, e, o, c, di, ta, amar, mo, dhe, shong, cromon, ho, che, tar, kach, te, ki, amar, kach, e, kras, mit, ho, e, jo, le, ar, che, e, j, no, a, mi, amar, e, tar, ho, nir, problem, ta, ho, che, ar, ho, dhaka, j, ki, j, kar ho, j, di, pur, be, te, ke, anxiety, disorder, otobar, depressi, disorder, te, ke, tha, ke, ho, e, ke, che, kane, che, ta, dhir, che, help, les, niz, bol, n, ho, business, bol, n, e, gul, sha, te, fight, korar, j, no, ta, dhir, ho, che, agh, e, odokom, coping skill, ta, tha, ke, na, ta, rakhum, falna, ne, bol, ho, e, jai, tar, mo, dhe, j, di, e, e, aggressive variety, of, o, c, dhita, j, di, add on, ho, e, to, fan, shita, ar, ho, bhi, distressing, ho, e, pa, ek, he, he, j, tar, a, ta, dhir, oi, j, aghir, j, tar, del, valna, bil, ti, chi, lo, ba, ta, dhir, j, weakness, gul, chi, lo, che, gul, sha, te, gul, j, no, ta, dhir, mo, dhe, j, distress, gul, ho, che, oi, distress, gul, sha, te, fight, korar, j, no, ta, del, phi, to, te, ke, eng, to, ne, ar, j, to, id, hoi, ba, aggression, to, id, hoi, ar, ita, j, t, k, komon, ni, amra, dhikhi, j, factor, mo, dhe, j, kono, ba, bi, j, di, tar, a, kono, trauma, tik, event, experience, kori, i, dar, ho, te, pa, rej, tar, a, fik, tim, kono, ba, bi, tar, abu, z, ho, e, j, od, ba, kono, j, valna, en, sir, tar, a, i, weakness, or, ta, tara, shakhi, ho, e, te, e, shomosto, factor, ta, del, raen, er, mo, te, jo, kono, kor, ba, kai, e, a, ta, por, je, dhikha, j, ta, del, mo, te, a, k, term, aggression, jo, le, ashe, i, bom, ita, j, de, ar, ho, te, tha, ke, e, aggression, ta, to, hon, dhikha, j, e, e, aggressivari, de, of, o, c, d, de, transform, hoi, a, kon, am i, j, ta, bol, bo, je, e, e, je, aggressivari, de, of, o, c, d, a, tar, ki, ki, karab, impact, e, ta, mano, shirmote, por, te, par, je, te, ho, ce, protamottu, je, tar, gross, functional, environment, ho, te, par, je, ki, ro, kon, je, tar, je, occupational, function, environment, ho, te, par, es, kui, performance, ball, and work, performance, ball, and shing, lo, te, impairment, ho, te, par, je, signifikent, li, dhikha, j, je, tar, inter, personal relationship, onig, bish, impairment, ho, je, onig, difficult, ho, e, par, e, dhada, j, no, acta, relationship, maintain, kora, kitre, je, to, j, ro, kon, relationship, ihog na keno, ami, amonu, de, kit, je, je, i, e, e, gress, verity, of, o, c, dhik, patient, as, je, kub, c, v, y, ro, b, stai, as, je, e, bom, ho, ce, je, tar, ba, cha, che, re, che, ala, dh, ba, cha, ni, niji, ki, isolate, kore, ra, k, ce, keno, tar, moni, ho, e, je, tar, kachir, mano, shir, di, ke, ta, ka, li, tar, moni, ho, e ki, jami, ta, ki, kutikore, fil, wo, ce, onig, par, lo, ba, je, ha, to, ba, cha, tar, ma, ho, ba, ba, ho, k, hai, ba, tar, spa, su, tar, shontan, ho, pan, ba, lo, ba, je, kintu, oi, ba, ita, ta, ki, e, to, b, je, wik, kore, fil, ce, sh, sh, shi, moni, kore, je, ami, ta, dhik, asipase, tha, kle, ta, dhik, j, ro, e, dh, riz ki, he, jat, ce, e, to, po, je, shi, ta, dhik, e, void, kore, a, shir, room, ala, da, kore, f, n, b, je, khani, ta, di, j, room, e, a, kata, kyo, ta, hote, pa, re, j, de, and, re, dhik, shi, ta, hote, sui, sa, dhil, thot, lag, gul, thik, shi, ja, kun, bhe, hote, pa, re, na, shi, ja, kun, kar, hil, cha, na, shi, ja, kun, ka, gun, prof, e, na, hil, po, na, na, niji, kyo, hil, kut, te, pa, re, na, to, kun, dhik, j, e, pa, re, chul, te, chul, te, chul, te, tar, mo, te, ti, bro, rokom, sui, sa, dhil, thot, chul, e, shi, e, kuna, mi, bo, bo, j, e, dhorn, e, khe, se, kibhabe, tritment, korah, hail, so, dhorn, e, ro, si, dhil, mo, toi, e, dhorn, e, ro, si, dhil, kitu, kawman, bi, sho, aj, j, ekki, petr, ne, tritment, korah, hail, j, mo, amra, psychotherapi, de, se, bhid, de, se, bhid, ki, shi, ta, ho, cha, koglitibi, bi, hir, thirapi, ek, han, ek, hail, ek, han, ek, han, ne, oi, patient, ir, je, negitif, ko, ni, shanta, negitif, typo, j, amra, me, je, chintakorah, je, petr, ne, tha, eta, ki, etbres, korah, hail, e, bo, shi, ta, khe, negitif, thot, gula, ke, positive, thot, de, replicment, korah, hail, or, tha, de, re-stracturing, korah, hail, shi, onu, prate, shi, je, bhid, ve, rgul, ko, cha, shegul, ha, kire, kire, kire, hail, tarpore ho, cha, medikation, ek, han, e, amra, ek, sa, sa, re, de, eta ki, ant i, ang, sa, eti, eti, sin, gula, de, eta ki, tarpore ho, cha, je, zail, hail, strategis, gula, oi, patient, ke, shi, khe, da, ah, hai, shegul, ki, je, ni, sh, jaman, mone, koran, je, relacation, technik, gula, sh, e, app, le, kor, ve, jo, han, jo, han, tar, mo, de, oning, bhid, de, stress, tori, ha, hai, de, breathing, exercise, kurt, de, pare, mindfulness, meditation, kurt, de, pare, tar, thot, distraction, ar, juna, ki, ki, koran, projul, shegul, shi, khe, da, ah, hai, chai, bhish, aigul, tar, juna, etchif, koran, koran, tar, juna, kub, easy, shai, bhish, aigul, takhe, list, kurt, de, bala, hai, je, mone, gol, sating, kurt, de, bala, hai, je, jegul, t, tarpore, etchif, koran, shom, hop, jegul, shom, hop, na, takhe, shi, te, imne, te, val, na, re, bhid, personal, cheer, acta, manu, sh, tar, mo, de, jegul, etchif, kurt, de, par, bhish, se, gul, jedi, shi, tar, get, koran, ta, hal, tar, distra, saru, pere, jabe, eta, shobar, kid, re, e, da, onir, gol, sating, koran, tal, dhalkar, tarpore, ho, j, acta, de, ni, hel, de, rut, de, maintain, kurt, de, ha, be, tarpore, tar, j, se, j, bhapare, par, o dhoshi, shi, j, par, o dhoshi, tar, jega, gul, ke, ar, shundar, kore, na, chur, kurt, de, ha, be, emong, se, gul, ke, practice, kurt, de, tat, eta, tar, mu, de, easy going approach, he, da, tar, chun, j, se, e, aki, ho, ch, par, o dhoshi, shi, da, ke, ar, o, be, shi, jedi, shi, flari, j, kurt, de, jai, tat, hal, eta, mu, de, ek, accomplishment, filt, kach, kor, be, tarpore, ho, j, take, shi, te, ha, be, j, tar, koon, koon factor, gul, take, trigger, kurt, shi, o, trigger, gul, ke, ar, intify, kore, shi, ono, jai, kid, kurt, te, ha, be, eta, o, take, shi, kye, da, ha, he, tapore, ar, ki, kor, jete, par, jete, ha, ke, emra, challenging thoughts, shi, kye, de, e, be, par, emi, ono, koono, epi, sure, inti, te, le, bol, bo, jete, challenging thoughts, take, kid, jin, ish, last, le, ho, ch, j, maintaining a positive attitude, jete, ho, kna, kya, no, o, ik, hottonar, jonu, take, positive attitude, tar, ak, de, ha, be, jai, ami, ay, take, negitiv, ha, be, ni, bona, negitiv, ha, be, bhab, bona, ami, eta, ke, positive, li, de, bo, emom, positive, ha, be, ama, ke, e, hottonar, tashatem, del, kurt, te, ha, be, ar, ki, ch, kye, ch, pesentir, kid, re, support, grubbe, involv, kor, ha, he, he, ch, li, ha, kya, amadir, u, war, all, ag, gris, if, variety, of, o, c, deer, dhukta, kich, hut, har,ona, asha, kuri, abh, nadir, kaji, ash, be,',
    'T__LifeSpring_3': 'ھا پلتے گی امت جبونٹے گتے پتے گتے تو یہ ششش پورٹ تھے کہ امت جبونٹے گی شنٹی پلی گتے تو یہ آچی بولی ایشامشہری آمیہ آچی نو تو تو بابارشتے شنگ شلکورا ایشامشتے گی کبی پلی جتا کی بیرنٹس در ادیشے مولو تو قطا گولو بولچھے کمان بولچھے تایتا کمان بورا رکوتا کارون اوٹر اوٹرستیشن امانا بیرنٹس را اوھ رفو پر ایشوی امت شنٹان دی رپوری دھانے توکسے کتا گولو چانل کرتا کی ایبون ایر کارونی امت شنٹان را کتر کو بیپورٹ جوستم امشکا بلس تایب دی جائے تایدر انڈر کنفیدن ٹیشی بے تویری کتی ایتا کتبورٹ بھمیکا اچھے ایبیپورگونا مدد ہوتت دھانا رو بائی تای اچھ کی کتا بولچھے بولوتت پرینٹن فرستشن ایبیشوی تینی فرستشن باہتشا کب امانلی ایک سپیرینز دیکٹر فیلنگ امان کنو جبون نہیں بیان کنو جبون پاہ جابنا حویتو جی جبون افرستشن نہیں رایت انت فرستشن ارسٹار سگونک انتوانک کتر ایتا کنو افیزکل ومنٹل سکنس اوتا پر کنو پر فیشنن وورکر کرم اوتا پر اپنر کالزگل ریلشنشی پے ایک چوٹ ارموائل اوار کنو کنو کنو اوتی رکتو دائتیٹ چاپ باہتشر فرستشن ایبیلی تزو اپنے که اوتا شمی فرستشن ایبیکے تیلی تیدی پر سورس با اوت شو ایفرستشن ایت جیتا ای اکنا گیا نو با جائی حکنا کنو امنے جو کن فرستشن ایت اکچی ایتا کن شاہ بایکن بیشو ایکیو او امرا شاہ بایک بھا بے ایک سبت کتس ترغل کنی چوٹ ایتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ ٹیکvery这是 اپzać ام hottest ام reperlem امام اگو اپسیٹوں امام اگو اکتا چالجون اپس Cute اescharest امام اگو اُوسکگری ا]: цیان اطلاع frustrating ابرام اگو را مشا County ایک تحşacka اجتا کنو اپسیٹی اکتا کنو اپسیٹو اپسیٹ Kazuya امس کتا کنو اپسیٹےhic جیتاکیمری بولچی ان جنگشنگشنگشنگ اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر ا اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگ اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگ اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگ اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگ اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگ اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگ اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگ اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگ اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگ اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگ اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگ اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگ اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگ',
    'T__DoctorolaTV_23': 'downt tastei.ṭhābhiashi.ṭhābhiashi.ṭhābhiashi.ṭhābhiashi.ṭhābhiashi.ṭhābhiashi.ṭhābhiashi.',
    'T__DoctorolaTV_24': 'With you again? Subscribe to our channel for more recipes and recipes such as 2012 Premiere Peace be upon you all my editor Feb Hansen consultant Thaaka Dharmaology Institute Beaglezer Tri we are going to do a rainy season with the sunlight now lets you know if Thaaka Dharmaology Institute will bring in a lot of rain or Wisconsin I am sure that the people who are in the hospital are not in the hospital yet. Today, I am going to visit the hospital. Today, I am going to visit the patients of the hospital. I am going to visit the patients of the hospital. I am going to visit Dr. Sharkar Mahambok Ambedshami. I am going to visit the patients of the medical advisor, Dr. Rabia Praswama. I am going to visit Dr. Humaira Bhushra from the Asthetic Ge毒 Burg Rest. I am going to visit the patients of the hospital and thank Dr. Shradhs expert for visitingе. It is a pleasant joy. I want to thank you very much to you. We are working on a YouTube channel and doing it on Facebook. We will be doing a lot of work on it. We will be doing a lot of work on it. I hope you like it. I hope you guys liked it. And dont forget to watch the next videos. Dont forget to like the video. Dont forget to subscribe. Thanks for watching this video. See you in the next video. See you in the next video. We are not going to be able to do anything. We are not going to be able to do anything. We are not going to be able to do anything. We are not going to be able to do anything. We are not going to be able to do anything. We are not going to be able to do anything. We are not going to be able to do anything. We are not going to be able to do anything. We are not going to be able to do anything. We are not going to be able to do anything. We are not going to be able to do anything. We are not going to be able to do anything. We are not going to be able to do anything. We are not going to be able to do anything. We are not going to be able to do anything. We are not going to be able to do anything. We are not going to be able to do anything. We are not going to be able to do anything. We are not going to be able to do anything. We are not going to be able to do anything. We are not going to be able to do anything. We are not going to be able to do anything. We are not going to be able to do anything. We are not going to be able to do anything. This experience brings us to thankfulness. We are not going to be able to do anythingrom then on the 댓 you should why are you highlighting your website as well? The same is done in the newspapers straight ahead of mine. So, when the artistic pleasure that ends there more thanly goes on And here is saying, if theres a demostrations working, we will need to help out a range of efforts. So disappoint and say that theres a bit of mistrust on your strips. So while youre talkingmpage of commercials, then, my office will be hanging out in the office. So sir, youre working? They are, youre working? Yes theyre doing market taxes, ones of all shows like this and all of them keep up. Yeah theres so much. So this huge list of  in this enterprise and so... actually, no matter how much they experience in our industry that theyre gonna be popular in this industry, Rメulouslydal Thank you a lot I wanted to show you a modern technology like what weve already done in the usual videos And what was your impression of it? Viveki BSOOF Our Geological arm wrestle, three gods has been killed by the feeling and defence of Afghanistan, the area at where Tehran bin wart Software called the details, this means and in itself because because of my topic, they are not going to let me imitate the mapping of abstract languages. so I have to improve the countries that ignore the whole state share the country. and it will be very important today. thank you all.',
    'T__DoctorolaTV_25': 'کی காட்ச்சத் கpage Cohbitдு hair ٹsoft ٹ наверfind ھاں میں جھے وارچوال دنیاں, یہ کہنا میں وہ نی کھون بیچھر ہونکوری, شیی وارچوال دنیاں, پروبیش کر وارچر نہ ہوا دے, دیوائیس کو لویی اسکتے ہاں, شے گلوں چکھر کوتی کر ہوتے کی نہ, شیٹھاں یہ, وہ نی کی کیونто چینتی تو آچھے, نی بوں, نی کے جنٹے چچھے, نی بوں, وہ نی کے آلرری دی, تر بھگتب ہوں گیاچھیں. امریکٹا کو تھا, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, ابھر کھیل کرنے, یہ دیوائز گولے, کمپیٹر گولوں جیروں میں تھاکے, شیروں گولہ تھے اور اسی ہو تھاکے, ابھر بیشی اسی تو تھاکہ شروع کرے, چیشتے ہیں تو, انترشش کو انوبرٹمنٹ, دوٹھم میں لے, چکھر پانی خوب طارہ طریقیں تو اورے جاتھے, پیرہ سپوٹھ روٹھے بہت کلوں, گیٹھ فلماس, نیڈوں کوئی بھاگی روٹھیک, ملٹپلون شاہو دیلے ار بششتوگیٹو لفس مارس, ویرس پوٹھ روٹھی دیاں. امریک چکھر پولوک پھلچی کم شوطرانک چکھے جی پانی بار بار پانیر ملیٹھا آبار اشپے مالا ٹھا چھولے جا بے نوطن پانی اشپیٹھا یہ کافر کھولے رگھے ایک ہٹھونہ طاکھن کم کرچے. ابھر دیلھا کھن جاکھن اپنی کچھے تاکیے تھاکھن, توکھن اپنے چکھر چکھر چکھوٹھو چکھوٹھو مصلگولا ٹھا چکھولا ٹھا چکھولا ٹھا چکھا امریک چکھن اگدام دورے ایک ہٹھونہ دورے ایک ہٹھونہ دورے ایک ہٹھونہ دورے ایک ہٹھونہ دورے ایک ہٹھونہ دورے ایک ہٹھونہ دورے ایک ہٹھونہ دورے ایک ہٹھونہ دورے ایک ہٹھونہ دورے ایک ہٹھونہ دورے ایک ہٹھونہ دورے ایک ہٹھونہ دورے ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہک gekớ सھیک Mékwa ٹھون ٹھونہ ٹھونہ ٹھونہ ٹھونہ ٹھون<|ur|> ٹھونہ ٹھونہ ٹھونہ ٹھونہ ٹھونہ ٺٹھونہ ٹھونہ ٹھونہ अप्वाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभ'
    # Add more entries matching the predicted file names...
}


# Folder containing the predicted transcripts
transcription_folder = 'transcriptions'  # Change this to the path of your predicted transcripts folder

# Calculate KPIs using the function
kpi_results = calculate_kpis(transcription_folder, reference_transcripts)

# Print KPI Results
print("KPI Results:", flush=True)
for kpi, value in kpi_results.items():
    print(f"{kpi.capitalize()}: {value:.4f}", flush=True)

KPI Results:
Wer: 76.9999
Cer: 11.9932
Rtf: 0.0001
Accuracy: -75.9999
Precision: 0.7311
Recall: 0.2689
F1_score: 0.2674
Deletion_rate: 0.0000
Insertion_rate: 164.2647


In [ ]:
# Example reference transcripts (stored in a dictionary variable)
reference_transcripts = {
    'T_LifeSpring_1': 'Қaatashime galo international mainstream hygiene de mainstream hygiene bol tashu lam reki bhochi 8 intashu le kubi gurutta purnu amade deshi jyodyo oha bejini shi te re aalachuna hoa na kintu masi ke shomuay pori chanata maintain kora masi ke shushas te jebapatta 8 intashu le kubi gurutta purnu ʻum dhiyajet tein shale mainstream hygiene de muil thein te itchidoeta jay make mainstream and normal fact of life by 2030 8 intashu le kubi ujta purnu ʻum dhiyajet tein shale māsik jyegte nāri riktu tun to shaba bhi khatu na e i i i i shtak shaba mute jukeli dāve ʻasule itke kyanate bhi gurutta dāgut se itke etta shādharun khatu na hinsi be manush keksab karatunu e i pitsunā shu te unik gurukaruna tse amade deshi be bhi nnu elakai nāri dhe kamra dekhi je shaba itua shule shaba nbabe privilits known shaba i shaba n shu ju shubhi dha pannab hādhar shādharun khali amade deshi je jyegte nāri dhe paird bhebhar kutta bātschen afford kutta bātschen tarfamli tse teke pozitivli shaba bhi dha ho tse jinistā kinda oning jyagrātshe vind shādharun anjule amade deshi je nāriya kindu maasik ki e bhebapat teneye kubakta shādharun tun nārn e vung tāne bhebapat teke mone kauranjie amade bhebapat tene kauranjie kauratha bhaul tepar buna at bhi jyegte kubkhara paktat tain e i shumaiteni alaqona kunu bhabi kaura ujitna e dhe jyegte dhe khadar je maasikahatta paird lag bheb poli bhebharia hādda baba upadjun kama aptake bholte bātschen na tini affustivod kutchen ibu ar aftur tse jinistā tini kubshaba bhi bhabi prokash kurtta bārana ibu e dhe jyegte hodse dhe khadar tse jitāra affustivod kaur kapur bhabharkutchen tula bhabharkutchen ibu shabchidu kho janojie kapur kunu dhe khadar jie bhar kaurin killed가락 trandehto sade turn Technologies pand bush tested jyegte baatzha cabar par clouds Warja maasikhi sube bliding tuther i 52finnidaf 53finnidafeliolution bachelor effective studies Massachusetts ھا پشو مراشنگیدی ایک تشنیش چالکوری دیکھون جی جو دی امرہ پوری چھانوں بھابیں ماشی کے شومتہ پانھ نہیں کری امر جیتی تنی شوچی تو نہ ہوئی ابیپر جیتے میں شاہدہ رون بھابیں نانی ایک تنی میی جو دیتر مارکی شوچھ پہے بلتنا پرے شیجو دی پیڑی جونوں تر بابا کے بلتنا پرے جی پہ با آفیسٹی کاشش ماما جونے تو پیڑی نیشو تا لیکن دو تارجونوں ایماشی کے شومتہ کو کاشتو کر ہوئی دارا ایبو امرہ ایت او شونی جو کنہ امرہ پیشندے کے بولی این گاٹیانہ بولن جی امرہ مامی تو کافر بابا کریں شامل کنو شامو شامی او بابا کر لیکن شامو شامی اشونی دیکھون شامو شاہدی تو امرہ نوتنوتنی جانتے پریبا شکتے پری ایبو ان دیکھا جی ایزی ایماشی کے شومائی پوری چھانوں تھا اٹتد مینسٹرال حاجین شامو شاہدی تو ایسے ایسیونہ امی شاموستو عجارت کی شوری جارا نوتن ماشیجا دیشورو ہوتشے بابا مہیشب اپنا دیجا مان دائکتو دیکھا ماشی کے بیپر گولے بوچی دا ایک بھابی ایماشیجتے جی کب شادہا نکتا گھتونہ ایتج پر اتکتی ناری جبونے ایتی نیدیشتو شامویت ہوای ماشیجا مدر شاریلی بھا بیک جون ناری تار کنو اوشوی تا تکینا دیکھا ماشیج اونیمی دکھولے اتھو ماشیگ نہ ہو لے اتھو اونےگ بیشیورے کنطام نچینتی تو ہو جی تر اٹھو کی ایت شوریر امین اکت جینی امین اکت گھتونہ جیت ماشیماشا ہون دیکھ جانان دائی جہا اما شوریشب سستمگلو شرنان مدتو چاہلچے اتبعی جینیشتے کب شاب بابی اگھا مہنو اکن دا کن اشوری جو گے دا پیپر اما ای بولشی نا ایتنی اپنے کلکولی اعلوچونہ کلون ایتا که ایمپورٹن ایک تا انوچونہ توپیک سوک کین تو کچھ شاہدھا اون بھابے ایک جون ناری تر ماشیک اشومت تار ایجیت ٹائمتا اییت ٹائمتا کیشی جانانکت شاہدھا اون اشومت ایشبی نیتا پری شیجنو تر فیم لیکچھے سپوچ چیتا پری شیجنو تر با با اطبع حزبین اطبع بہایتا که بولت پر جامر پید لگ بئینتا ایوین امدی سکون کلکولیج کلوتے بھی نا جا گئی اماشیگی شو مائی پسانیشری پیدر جون جو دی کونسستم راکھا جائے ایجی اام را جتا اچھی جی اام راک کون کون بچاکا دی شیمیشن تتک امنا پیکٹنی تی پاچھی شیتا کین تو امدیشن اارو بھالو ایو پس پسا ایت گلت پر نو بیشا شیتا اوچھی ایوین امدیشن ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایو surgeries ,یوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایو une ایوی dieser ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوین ایوzilیوین ایوین ایوین ایوین ایوین ایوین ایوین مع ایوون ایو',
    'T_LifeSpring_2': 'Amra, aeportjunctu, onigrokomir o cd, theme, but content, bapari amra ishune chtainu, jhe mon jiba no de kreena pae, sapi chunapak mo nehae, aeko tamatai to klesha hojibbir ho te chana. Innes ni ye shara kondakha je je intrusive thought ho de tha ke, dar pre ho chhe je sexual type of content frequently matam mo te ashe. Ae golo bapari amra ki to kongbe si bhalui jani tainu. Ae kia mi, ae reig dharo ne o cd veritine ye kichu kata bol bu, shaita ho chhe je agressive variety of OCD. Ita ki jinish, dakin nami ho chhe amra to dharo napat chtainu je agressive variety. Ki ho ye kane je oi partikular patiente mo nehae tor teke, ekthana aggrishen kach ko re, ki rokom aggrishen. Ita kindu kubi asole distressing e bom disrap deep type. Ki ho ye kane. Ammi asole ae gokom patientke, onig bhi si manoshig habe jontrona pete de ki. Onig bhi kosh te tha ke, onig worried tha ke, onig angzras tha ke. Ki ho ye, dhaka jai ke, ce oi partikular patiente mo nekori ki, ce amar madhu ne, amar kachir manushamar puri barer konon log jon, amar konu friend, athaba ho chhe bilavit konu person ke, ammi khote kore feldepari. Onig bhi si rokomir haam full ae kichu ghoti hai feldepari, jar karneshie, dhur kata nae, putitu ho te pari, athaba ho chhe je she even marau jete pari. Ae jin bhoi da, je ita tar muthe kach kore ke na je she kindu, jani je ami chai le, kindu tar khote kore feldepari. Ae bhoi da, kindu, e bhoi da ita ar muthe kore kore kore kai, je she shara konu jani wori takin, barer ni jir upothe ke, sheta niyum chon hari feldepari, jikunu shama shhe, ae munekta arj fil kore abar bhoi opai, jar amar tanataru khote kore jete pari. Ae jee ae ta, lup, e lupir mo de, oi be chara ba be chari, ghor pa kete tha ke, e bhoi na ko rakumir tha tha ke, je supris kore kje no, banu jo kore kje no, aning bhi shye mental iford de, ae visha saikel ta, roti din, roti dinyotu tar muthe, tine bhi shilbhak shomuay ghotte tha ke, e bung da kha jai, je uiter fale, oi be chai, aning bhi shi wori takin, e bhoi anjas tha ke, je je agrisi variety of OCD, eta tine torneer hoe tha ke, tine torne bhoi de, tine torneer, tine hoe tha ke, bhi bhi no gavisyonay da kha jai, shite kiki je, harming aadars, eta kiro kum, e jai kto aage jeta bholam, shi chintakorikje, amar ghorre manush ke ami jai kuno shomuay myri fil te pari, kono chhuri dikle tar mone hoi ki, ja ami akhan haudibat, amar shamne manushta ke, stebh kore fil te pari, tar diya shlai dikle mone hoi ki, ja ami ghorre bhi tar agun lagi fil te pari, hae ba ami amar shamne manushta gae agun lagi di te pari, eidhaar onir bhoi ta tar muthe kach kurte tha ke, shi gintakasule kore fil le na, hae ba, kintu shi oibhoi ta tar muthe kach kurikje, ami amar kontri ni jero po kontrulta ke hari fil te pari, ebong ami ita chaili purte pari, ae je, thinking termode, etaar prosister mode shi gurthe tha ke, ditio je pattern da shita ho che, je, hari fik, ebong ho che, dinjara stebh of je, e mei jes gulho, at kalpona jitra gulho, tar chokhi shamne bha shte tha ke, abar tari, disease prosister karo ne, shi chok bontho kore priparasi ni, kaalpona kurthe tha ke, je, shi tar, kachil kono manushke, ba, tar puri barel logjon ke, ba, tar friend huk, je, huk karo, che kote kore fil te pari, ki, rokom shi kaalpona kurthe tha ke, ja, ami, ami, ami gammar kono friendesha de, ami, roth kraskurt shi, e shamne ami, oke, dhakkadi a, ami fil le dilam, roth er mode hai, ek tan gari aur hupudde, uthe chole ja, eidhanir, thot termode, ashtethe ke, thot ki, shi kaalpona kore, abar ki ashe, irkomu dhakkha ja, je, no tun ke, ma, huye, che, shi tar, bebi ke, chole fil te, beba, che, tar, bebi ke, ami, abe, che, pe, durbe, ja, na, bebi tar, ek tan, kankhi to, ba, aur pruttashi to, ba, aur aur akumir, konu, dur khatunak hurthe ja, abar karo, karo, khi, to, irkomu dhakkha ja, je, tar, ashust, to, ba, ba, ba, ba, ba, ek dung, bea, chanae, poria, se, mu, mu, shawbos, tha, ta, dhe, ke, jo, shi, mo, ne, kore, ki, jami, ba, list, cha, padhe, mere, fili, ma, e, rokom, thot, ne, o, kin, tu, ashe, ma, prochon, oye, be, charabab, be, chari, ashole, prochon, no, rokomir, mentally, distress, ni, ashe, jami, tu, itas, le, kurthe, cha, i, na, kin, tamar, be, dur, itas, chole, ashust, shi, tri, tu, je, petan, shi, tu, je, doing something, embarrassing, but illegal. shi, tu, ki, rokom, je, ashole, shi, ka, ki, opuman, kurthe, cha, tine, kin, tu, hoot, kore, kono rokomir, kono, prohokir, kasi, chara, shi, tar, sham, ne, raktum, manu, shi, a, jo, ta, i, shi, opuman, kore, fil, shi, na, to, opuman, shi, a, kurthe, to, bha, shai, shi, ga, legala, chat, ba, kore, di, jo, ma, emun, embarrassing, ki, chuk, kore, fili, j, ta, pore, koni, ta, lo, kachi, ma, ne, ho, chai, ta, amiku, bha, ember, s, fil, kurthe, bom, j, kye, shi, tar, ne, kono, kata, boli, fili, shi, da, kha, j, j, embarrassing, fil, kore, i, je, petan, gullu, amiku, boli, ha, j, aj, aj, aj, tine, gullu, eg, gullu, patient, kin, tu, emun, bha, na, j, tar, bha, v, v, j, tar, kono, violence, sir, kono, iti, hash, aj, aj, e, ja, ho, chat, kore, i, shul, ho, shi, adris, kurthe, pare, j, amar, e, je, thart, gullu, ash, che, eg, gullu, o, proje, ni, ho, j, chit, ho, o, he, tu, emun, amar, j, ho, te, pare, e, bom, ordnir, j, no, rish, ge, ho, te, pare, shi, eg, gullu, adris, te, ke, kurthe, pare, k, n, tu, shi, ait, a, te, ke, bhe, rote, chai, ho, k, n, tu, shi, ait, a, te, ke, bhe, rote, pare, na, on, he, bhe, rote, ge, ar, ch, fil, kore, i, tai, ho, te, ge, kore, j, dhar, on, adr, ar, e, jo, no, patient, kikore, j, bhe, bhe, nor, akomir, j, thart, gullu, ni, a, che, tar, e, thart, gullu, k, che, on, he, replace, korar, che, stakore, e, bom, shi, eg, la, ke, sap, pris, korar, che, stakore, abar ho, che, on, he, bhe, rote, pare, ho, te, pare, ho, te, na, e, bom, ordnir, j, ni, ne, j, ke, no, ash, li, bha, most commonly, dhaka, j, j, j, n, tika, j, influence, sheta, to, a, che, tar, pore, dhaka, j, oi, particula, patient, ter, j, di, friend, per, family, nir, kar, mo, dhe, j, di, o, c, di, tha, ke, tha, ke, topon, dhaka, j, che, ch, ta, kor, e, bhe, korar, e, kore, j, amar, j, bila, ve, tar, ho, bha, ma, family, m, ba, frender, j, o, c, di, ta, a, che, e, o, c, di, ta, amar, mo, dhe, shong, cromon, ho, che, tar, kach, te, ki, amar, kach, e, kras, mit, ho, e, jo, le, ar, che, e, j, no, a, mi, amar, e, tar, ho, nir, problem, ta, ho, che, ar, ho, dhaka, j, ki, j, kar ho, j, di, pur, be, te, ke, anxiety, disorder, otobar, depressi, disorder, te, ke, tha, ke, ho, e, ke, che, kane, che, ta, dhir, che, help, les, niz, bol, n, ho, business, bol, n, e, gul, sha, te, fight, korar, j, no, ta, dhir, ho, che, agh, e, odokom, coping skill, ta, tha, ke, na, ta, rakhum, falna, ne, bol, ho, e, jai, tar, mo, dhe, j, di, e, e, aggressive variety, of, o, c, dhita, j, di, add on, ho, e, to, fan, shita, ar, ho, bhi, distressing, ho, e, pa, ek, he, he, j, tar, a, ta, dhir, oi, j, aghir, j, tar, del, valna, bil, ti, chi, lo, ba, ta, dhir, j, weakness, gul, chi, lo, che, gul, sha, te, gul, j, no, ta, dhir, mo, dhe, j, distress, gul, ho, che, oi, distress, gul, sha, te, fight, korar, j, no, ta, del, phi, to, te, ke, eng, to, ne, ar, j, to, id, hoi, ba, aggression, to, id, hoi, ar, ita, j, t, k, komon, ni, amra, dhikhi, j, factor, mo, dhe, j, kono, ba, bi, j, di, tar, a, kono, trauma, tik, event, experience, kori, i, dar, ho, te, pa, rej, tar, a, fik, tim, kono, ba, bi, tar, abu, z, ho, e, j, od, ba, kono, j, valna, en, sir, tar, a, i, weakness, or, ta, tara, shakhi, ho, e, te, e, shomosto, factor, ta, del, raen, er, mo, te, jo, kono, kor, ba, kai, e, a, ta, por, je, dhikha, j, ta, del, mo, te, a, k, term, aggression, jo, le, ashe, i, bom, ita, j, de, ar, ho, te, tha, ke, e, aggression, ta, to, hon, dhikha, j, e, e, aggressivari, de, of, o, c, d, de, transform, hoi, a, kon, am i, j, ta, bol, bo, je, e, e, je, aggressivari, de, of, o, c, d, a, tar, ki, ki, karab, impact, e, ta, mano, shirmote, por, te, par, je, te, ho, ce, protamottu, je, tar, gross, functional, environment, ho, te, par, je, ki, ro, kon, je, tar, je, occupational, function, environment, ho, te, par, es, kui, performance, ball, and work, performance, ball, and shing, lo, te, impairment, ho, te, par, je, signifikent, li, dhikha, j, je, tar, inter, personal relationship, onig, bish, impairment, ho, je, onig, difficult, ho, e, par, e, dhada, j, no, acta, relationship, maintain, kora, kitre, je, to, j, ro, kon, relationship, ihog na keno, ami, amonu, de, kit, je, je, i, e, e, gress, verity, of, o, c, dhik, patient, as, je, kub, c, v, y, ro, b, stai, as, je, e, bom, ho, ce, je, tar, ba, cha, che, re, che, ala, dh, ba, cha, ni, niji, ki, isolate, kore, ra, k, ce, keno, tar, moni, ho, e, je, tar, kachir, mano, shir, di, ke, ta, ka, li, tar, moni, ho, e ki, jami, ta, ki, kutikore, fil, wo, ce, onig, par, lo, ba, je, ha, to, ba, cha, tar, ma, ho, ba, ba, ho, k, hai, ba, tar, spa, su, tar, shontan, ho, pan, ba, lo, ba, je, kintu, oi, ba, ita, ta, ki, e, to, b, je, wik, kore, fil, ce, sh, sh, shi, moni, kore, je, ami, ta, dhik, asipase, tha, kle, ta, dhik, j, ro, e, dh, riz ki, he, jat, ce, e, to, po, je, shi, ta, dhik, e, void, kore, a, shir, room, ala, da, kore, f, n, b, je, khani, ta, di, j, room, e, a, kata, kyo, ta, hote, pa, re, j, de, and, re, dhik, shi, ta, hote, sui, sa, dhil, thot, lag, gul, thik, shi, ja, kun, bhe, hote, pa, re, na, shi, ja, kun, kar, hil, cha, na, shi, ja, kun, ka, gun, prof, e, na, hil, po, na, na, niji, kyo, hil, kut, te, pa, re, na, to, kun, dhik, j, e, pa, re, chul, te, chul, te, chul, te, tar, mo, te, ti, bro, rokom, sui, sa, dhil, thot, chul, e, shi, e, kuna, mi, bo, bo, j, e, dhorn, e, khe, se, kibhabe, tritment, korah, hail, so, dhorn, e, ro, si, dhil, mo, toi, e, dhorn, e, ro, si, dhil, kitu, kawman, bi, sho, aj, j, ekki, petr, ne, tritment, korah, hail, j, mo, amra, psychotherapi, de, se, bhid, de, se, bhid, ki, shi, ta, ho, cha, koglitibi, bi, hir, thirapi, ek, han, ek, hail, ek, han, ek, han, ne, oi, patient, ir, je, negitif, ko, ni, shanta, negitif, typo, j, amra, me, je, chintakorah, je, petr, ne, tha, eta, ki, etbres, korah, hail, e, bo, shi, ta, khe, negitif, thot, gula, ke, positive, thot, de, replicment, korah, hail, or, tha, de, re-stracturing, korah, hail, shi, onu, prate, shi, je, bhid, ve, rgul, ko, cha, shegul, ha, kire, kire, kire, hail, tarpore ho, cha, medikation, ek, han, e, amra, ek, sa, sa, re, de, eta ki, ant i, ang, sa, eti, eti, sin, gula, de, eta ki, tarpore ho, cha, je, zail, hail, strategis, gula, oi, patient, ke, shi, khe, da, ah, hai, shegul, ki, je, ni, sh, jaman, mone, koran, je, relacation, technik, gula, sh, e, app, le, kor, ve, jo, han, jo, han, tar, mo, de, oning, bhid, de, stress, tori, ha, hai, de, breathing, exercise, kurt, de, pare, mindfulness, meditation, kurt, de, pare, tar, thot, distraction, ar, juna, ki, ki, koran, projul, shegul, shi, khe, da, ah, hai, chai, bhish, aigul, tar, juna, etchif, koran, koran, tar, juna, kub, easy, shai, bhish, aigul, takhe, list, kurt, de, bala, hai, je, mone, gol, sating, kurt, de, bala, hai, je, jegul, t, tarpore, etchif, koran, shom, hop, jegul, shom, hop, na, takhe, shi, te, imne, te, val, na, re, bhid, personal, cheer, acta, manu, sh, tar, mo, de, jegul, etchif, kurt, de, par, bhish, se, gul, jedi, shi, tar, get, koran, ta, hal, tar, distra, saru, pere, jabe, eta, shobar, kid, re, e, da, onir, gol, sating, koran, tal, dhalkar, tarpore, ho, j, acta, de, ni, hel, de, rut, de, maintain, kurt, de, ha, be, tarpore, tar, j, se, j, bhapare, par, o dhoshi, shi, j, par, o dhoshi, tar, jega, gul, ke, ar, shundar, kore, na, chur, kurt, de, ha, be, emong, se, gul, ke, practice, kurt, de, tat, eta, tar, mu, de, easy going approach, he, da, tar, chun, j, se, e, aki, ho, ch, par, o dhoshi, shi, da, ke, ar, o, be, shi, jedi, shi, flari, j, kurt, de, jai, tat, hal, eta, mu, de, ek, accomplishment, filt, kach, kor, be, tarpore, ho, j, take, shi, te, ha, be, j, tar, koon, koon factor, gul, take, trigger, kurt, shi, o, trigger, gul, ke, ar, intify, kore, shi, ono, jai, kid, kurt, te, ha, be, eta, o, take, shi, kye, da, ha, he, tapore, ar, ki, kor, jete, par, jete, ha, ke, emra, challenging thoughts, shi, kye, de, e, be, par, emi, ono, koono, epi, sure, inti, te, le, bol, bo, jete, challenging thoughts, take, kid, jin, ish, last, le, ho, ch, j, maintaining a positive attitude, jete, ho, kna, kya, no, o, ik, hottonar, jonu, take, positive attitude, tar, ak, de, ha, be, jai, ami, ay, take, negitiv, ha, be, ni, bona, negitiv, ha, be, bhab, bona, ami, eta, ke, positive, li, de, bo, emom, positive, ha, be, ama, ke, e, hottonar, tashatem, del, kurt, te, ha, be, ar, ki, ch, kye, ch, pesentir, kid, re, support, grubbe, involv, kor, ha, he, he, ch, li, ha, kya, amadir, u, war, all, ag, gris, if, variety, of, o, c, deer, dhukta, kich, hut, har,ona, asha, kuri, abh, nadir, kaji, ash, be,',
    'T_LifeSpring_3': 'ھا پلتے گی امت جبونٹے گتے پتے گتے تو یہ ششش پورٹ تھے کہ امت جبونٹے گی شنٹی پلی گتے تو یہ آچی بولی ایشامشہری آمیہ آچی نو تو تو بابارشتے شنگ شلکورا ایشامشتے گی کبی پلی جتا کی بیرنٹس در ادیشے مولو تو قطا گولو بولچھے کمان بولچھے تایتا کمان بورا رکوتا کارون اوٹر اوٹرستیشن امانا بیرنٹس را اوھ رفو پر ایشوی امت شنٹان دی رپوری دھانے توکسے کتا گولو چانل کرتا کی ایبون ایر کارونی امت شنٹان را کتر کو بیپورٹ جوستم امشکا بلس تایب دی جائے تایدر انڈر کنفیدن ٹیشی بے تویری کتی ایتا کتبورٹ بھمیکا اچھے ایبیپورگونا مدد ہوتت دھانا رو بائی تای اچھ کی کتا بولچھے بولوتت پرینٹن فرستشن ایبیشوی تینی فرستشن باہتشا کب امانلی ایک سپیرینز دیکٹر فیلنگ امان کنو جبون نہیں بیان کنو جبون پاہ جابنا حویتو جی جبون افرستشن نہیں رایت انت فرستشن ارسٹار سگونک انتوانک کتر ایتا کنو افیزکل ومنٹل سکنس اوتا پر کنو پر فیشنن وورکر کرم اوتا پر اپنر کالزگل ریلشنشی پے ایک چوٹ ارموائل اوار کنو کنو کنو اوتی رکتو دائتیٹ چاپ باہتشر فرستشن ایبیلی تزو اپنے که اوتا شمی فرستشن ایبیکے تیلی تیدی پر سورس با اوت شو ایفرستشن ایت جیتا ای اکنا گیا نو با جائی حکنا کنو امنے جو کن فرستشن ایت اکچی ایتا کن شاہ بایکن بیشو ایکیو او امرا شاہ بایک بھا بے ایک سبت کتس ترغل کنی چوٹ ایتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ اکتا کنو اپسیٹیگ ٹیکvery这是 اپzać ام hottest ام reperlem امام اگو اپسیٹوں امام اگو اکتا چالجون اپس Cute اescharest امام اگو اُوسکگری ا]: цیان اطلاع frustrating ابرام اگو را مشا County ایک تحşacka اجتا کنو اپسیٹی اکتا کنو اپسیٹو اپسیٹ Kazuya امس کتا کنو اپسیٹےhic جیتاکیمری بولچی ان جنگشنگشنگشنگ اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر ا اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگ اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگ اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگ اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگ اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگ اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگ اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگ اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگ اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگ اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگ اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگ اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگ اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگر اگ',
    'T_DoctorolaTV_23': 'downt tastei.ṭhābhiashi.ṭhābhiashi.ṭhābhiashi.ṭhābhiashi.ṭhābhiashi.ṭhābhiashi.ṭhābhiashi.',
    'T_DoctorolaTV_24': 'With you again? Subscribe to our channel for more recipes and recipes such as 2012 Premiere Peace be upon you all my editor Feb Hansen consultant Thaaka Dharmaology Institute Beaglezer Tri we are going to do a rainy season with the sunlight now lets you know if Thaaka Dharmaology Institute will bring in a lot of rain or Wisconsin I am sure that the people who are in the hospital are not in the hospital yet. Today, I am going to visit the hospital. Today, I am going to visit the patients of the hospital. I am going to visit the patients of the hospital. I am going to visit Dr. Sharkar Mahambok Ambedshami. I am going to visit the patients of the medical advisor, Dr. Rabia Praswama. I am going to visit Dr. Humaira Bhushra from the Asthetic Ge毒 Burg Rest. I am going to visit the patients of the hospital and thank Dr. Shradhs expert for visitingе. It is a pleasant joy. I want to thank you very much to you. We are working on a YouTube channel and doing it on Facebook. We will be doing a lot of work on it. We will be doing a lot of work on it. I hope you like it. I hope you guys liked it. And dont forget to watch the next videos. Dont forget to like the video. Dont forget to subscribe. Thanks for watching this video. See you in the next video. See you in the next video. We are not going to be able to do anything. We are not going to be able to do anything. We are not going to be able to do anything. We are not going to be able to do anything. We are not going to be able to do anything. We are not going to be able to do anything. We are not going to be able to do anything. We are not going to be able to do anything. We are not going to be able to do anything. We are not going to be able to do anything. We are not going to be able to do anything. We are not going to be able to do anything. We are not going to be able to do anything. We are not going to be able to do anything. We are not going to be able to do anything. We are not going to be able to do anything. We are not going to be able to do anything. We are not going to be able to do anything. We are not going to be able to do anything. We are not going to be able to do anything. We are not going to be able to do anything. We are not going to be able to do anything. We are not going to be able to do anything. We are not going to be able to do anything. This experience brings us to thankfulness. We are not going to be able to do anythingrom then on the 댓 you should why are you highlighting your website as well? The same is done in the newspapers straight ahead of mine. So, when the artistic pleasure that ends there more thanly goes on And here is saying, if theres a demostrations working, we will need to help out a range of efforts. So disappoint and say that theres a bit of mistrust on your strips. So while youre talkingmpage of commercials, then, my office will be hanging out in the office. So sir, youre working? They are, youre working? Yes theyre doing market taxes, ones of all shows like this and all of them keep up. Yeah theres so much. So this huge list of  in this enterprise and so... actually, no matter how much they experience in our industry that theyre gonna be popular in this industry, Rメulouslydal Thank you a lot I wanted to show you a modern technology like what weve already done in the usual videos And what was your impression of it? Viveki BSOOF Our Geological arm wrestle, three gods has been killed by the feeling and defence of Afghanistan, the area at where Tehran bin wart Software called the details, this means and in itself because because of my topic, they are not going to let me imitate the mapping of abstract languages. so I have to improve the countries that ignore the whole state share the country. and it will be very important today. thank you all.',
    'T_DoctorolaTV_25': 'کی காட்ச்சத் கpage Cohbitдு hair ٹsoft ٹ наверfind ھاں میں جھے وارچوال دنیاں, یہ کہنا میں وہ نی کھون بیچھر ہونکوری, شیی وارچوال دنیاں, پروبیش کر وارچر نہ ہوا دے, دیوائیس کو لویی اسکتے ہاں, شے گلوں چکھر کوتی کر ہوتے کی نہ, شیٹھاں یہ, وہ نی کی کیونто چینتی تو آچھے, نی بوں, نی کے جنٹے چچھے, نی بوں, وہ نی کے آلرری دی, تر بھگتب ہوں گیاچھیں. امریکٹا کو تھا, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, وہ نی کے آلرری دنیاں, ابھر کھیل کرنے, یہ دیوائز گولے, کمپیٹر گولوں جیروں میں تھاکے, شیروں گولہ تھے اور اسی ہو تھاکے, ابھر بیشی اسی تو تھاکہ شروع کرے, چیشتے ہیں تو, انترشش کو انوبرٹمنٹ, دوٹھم میں لے, چکھر پانی خوب طارہ طریقیں تو اورے جاتھے, پیرہ سپوٹھ روٹھے بہت کلوں, گیٹھ فلماس, نیڈوں کوئی بھاگی روٹھیک, ملٹپلون شاہو دیلے ار بششتوگیٹو لفس مارس, ویرس پوٹھ روٹھی دیاں. امریک چکھر پولوک پھلچی کم شوطرانک چکھے جی پانی بار بار پانیر ملیٹھا آبار اشپے مالا ٹھا چھولے جا بے نوطن پانی اشپیٹھا یہ کافر کھولے رگھے ایک ہٹھونہ طاکھن کم کرچے. ابھر دیلھا کھن جاکھن اپنی کچھے تاکیے تھاکھن, توکھن اپنے چکھر چکھر چکھوٹھو چکھوٹھو مصلگولا ٹھا چکھولا ٹھا چکھولا ٹھا چکھا امریک چکھن اگدام دورے ایک ہٹھونہ دورے ایک ہٹھونہ دورے ایک ہٹھونہ دورے ایک ہٹھونہ دورے ایک ہٹھونہ دورے ایک ہٹھونہ دورے ایک ہٹھونہ دورے ایک ہٹھونہ دورے ایک ہٹھونہ دورے ایک ہٹھونہ دورے ایک ہٹھونہ دورے ایک ہٹھونہ دورے ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہ ایک ہٹھونہک gekớ सھیک Mékwa ٹھون ٹھونہ ٹھونہ ٹھونہ ٹھونہ ٹھون<|ur|> ٹھونہ ٹھونہ ٹھونہ ٹھونہ ٹھونہ ٺٹھونہ ٹھونہ ٹھونہ अप्वाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभाभ'
    # Add more entries matching the predicted file names...
}
